# Experimentally Upload Jupyter Cell to Imgur

The code below uses a JS library called [html2canvas](http://html2canvas.hertzen.com) to take a screenshot of a Jupyter cell. The screenshot is then uploaded to Imgur via its upload API. It is activated by the keyboard shortcut `r`.

This is designed to work on Jupyter Notebooks version 4.0 or higher.

First we load html2canvas:

In [13]:
%%html

<script src="http://html2canvas.hertzen.com/build/html2canvas.js" charset="utf-8"></script>

In [14]:
%%javascript

// Next define a function for uploading the canvas to Imgur

// 14c0d864490a649
var ImgurScreenshot = function(api_key){
    this.imgur_api_client_key = api_key;
};

ImgurScreenshot.prototype.handleEvent = function(event) {
    // Get selected cells - easier to use jQuery here.
    var selectedCells = jQuery(".selected.cell");
    // Now we want to generate a canvas element for each selected cell.
    // Since it uses promises, it's not necessarily deterministic, so we keep track.
    this.canvasElements = Array(selectedCells.length);
    // Since this is async (yay?) we need to keep track of when we're ready for the next stage.
    this.canvasStatus = Array(selectedCells.length).fill(false);
    $this = this;
    selectedCells.map(function(index, element){ 
        html2canvas(element, {
            onrendered: function(renderedCanvas){
                $this.canvasElements[index] = renderedCanvas;
            },
            allowTaint: true,
            //height: jQuery(element).height()
        });
        // Set out tracker to "true".
        $this.canvasStatus[index] = true; 
    });

    // Set up a function to check that all of our status is done, every 100 ms.
    this.canvasTimer = setInterval(function(){
        if($this.canvasStatus.reduce(function(x, y){return (x * y);})) {
            $this.drawTogether();
            clearInterval($this.canvasTimer);
        }
    }, 100); 
}

ImgurScreenshot.prototype.drawTogether = function(){
    canvasElements = this.canvasElements;
    // Once we have all of the cells, we want to make a new canvas combining all of them.
    var height = canvasElements.map(function(x){return(x.height);}).reduce(function(x,y){return(x + y);}),
        width = Math.max.apply(null, canvasElements.map(function(x){return(x.width);}));

    // Now we create a new canvas element that's large enough.
    var finalCanvas = document.createElement('canvas');
    finalCanvas.height = height;
    finalCanvas.width = width;
    var canvasContext = finalCanvas.getContext('2d');
    // Look through and draw.
    var offsetHeight = 0;
    for(i=0; i < canvasElements.length; i++){
        var currentImage = canvasElements[i];
        canvasContext.drawImage(currentImage, 0, offsetHeight);
        offsetHeight += currentImage.height;
    }
    this.upload(finalCanvas);
}

ImgurScreenshot.prototype.upload = function(finalCanvas){
    var img = finalCanvas.toDataURL("image/png").split(",")[1];
    $.ajax({
        url: 'https://api.imgur.com/3/image',
        type: 'post',
        headers: {
            Authorization: 'Client-ID ' + this.imgur_api_client_key
        },
        data: {
            image: img
        },
        dataType: 'json',
        success: function(response) {
            if (response.success) {
                // Perhaps do something else with this - 
                console.log(response.data.link);
            }
        }
    }).error(function() {
        alert('Could not reach api.imgur.com.');
        w.close();
    });
}

// Define `r` as the keyboard shortcut

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help: 'Upload cell to Imgur',
    help_index: 'Upload a screenshot of the current cell and output to Imgur.',
    handler: function(x){ i = new ImgurScreenshot("14c0d864490a649"); i.handleEvent(x);}
});

<IPython.core.display.Javascript object>

Now try to run the cell below and press the `r` button. If everything works, a new tab will pop open with the screenshot. It should look like this

![](http://i.imgur.com/m3ZfPLB.png)

In [8]:
1+1

2

In [15]:
2+2

4

Multiple tabs! But it doesn't always work...

![](http://i.imgur.com/q1mJaun.png)